In [1]:
import dffc
from dffc.holt_winters import HWOptimizer
import os
import pandas as pd

In [2]:
codes = ['007467', '004253']
names = ['HL', 'GD']
start_date = '2019-07-01'
# end_date = '2025-07-01'

## Download & Update Fund Data
print("=" * 15 + "Start Process" + "=" * 15)
print("- Downloading fund data...")
try:
    fund_data = dffc.FundData.load('./data/hlvsgd_all.pkl')
except FileNotFoundError:
    fund_data = dffc.FundData.download(
        codes,
        names=names,
        # start=start_date,     
        # end=end_date
    )
    os.makedirs('./data', exist_ok=True)
    fund_data.save('./data/hlvsgd_all.pkl')

print("- Updating fund data...")
fund_data = fund_data.update()
# fund_data.save('./data/hlvsgd_all.pkl')
price_data_org = fund_data.get('unit_value')

## Append Estimate Data
print("- Appending estimate data...")
estimate_p = dffc.FundEstimateProvider()

price_data = dffc.append_estimates_to_prices(
    price_data_org,
    codes,
    estimate_p,
    target_timezone=None,
)

===============Start Process===============
- Downloading fund data...
- Updating fund data...
- Appending estimate data...
Estimate data fetched for 007467 on 2025-11-28 11:30:00+08:00
Estimate data fetched for 004253 on 2025-11-28 12:52:00+08:00
Added estimate rows for dates: 2025-11-28.
- Appending estimate data...
Estimate data fetched for 007467 on 2025-11-28 11:30:00+08:00
Estimate data fetched for 004253 on 2025-11-28 12:52:00+08:00
Added estimate rows for dates: 2025-11-28.


In [4]:
# 使用 HWOptimizer 类进行优化
optimizer = HWOptimizer(price_data)
hw_summary = optimizer.optimize(
    save=True,
    output_base_dir="./hw_opt_results",
    result_filename="demo_hw",
    max_workers=8,
    end_day_list=list(range(-400, 0, 40)) + [None]
)

 007467 processed! Alpha=0.0940, Beta=0.0000, Gamma=0.0510, Season=24.0


 004253 processed! Alpha=0.0981, Beta=0.0061, Gamma=0.4108, Season=16.0


In [6]:
# 使用类方法绘图
fundcode = '004253'
asset = optimizer.get_asset(fundcode)

fig = asset.plot(
    save=True,
    display=True,
    output_dir=f"./hw_opt_results/{fundcode}",
    filename_prefix=f"demo_plot_{fundcode}",
)

In [7]:
# 查看优化器状态
print(f"已优化资产数量: {len(optimizer.assets)}")
print(f"资产代码: {list(optimizer.assets.keys())}")


已优化资产数量: 2
资产代码: ['007467', '004253']


In [8]:
# 使用内置的 analyze_stability 方法分析参数
for code in optimizer.assets:
    asset = optimizer.get_asset(code)
    
    print(f"\n{'='*50}")
    print(f"=== {code} 参数分析 ===")
    print(f"{'='*50}")
    
    # 自动分析并获取推荐配置
    recommended = asset.analyze_stability(recent_threshold=-100)
    
    if recommended:
        print(f"\n>>> 推荐配置:")
        print(f"    Season Length: {recommended['season_length']}")
        print(f"    Alpha: {recommended['alpha']:.4f}")
        print(f"    Beta:  {recommended['beta']:.4f}")
        print(f"    Gamma: {recommended['gamma']:.4f}")
        print(f"    MSE:   {recommended['mse']:.6f}")
        
        print(f"\n>>> 稳定性分析:")
        analysis = recommended['analysis']
        print(f"    分析窗口: end_day >= {analysis['recent_threshold']}")
        print(f"    最常出现的 Season Length: {analysis['season_mode']}")
        print(f"    该配置在窗口内出现次数: {analysis['stable_count']}")
        print(f"    窗口内 Alpha 均值: {analysis['alpha_mean']:.4f}")
        print(f"    窗口内 Beta 均值: {analysis['beta_mean']:.4f}")
        print(f"    窗口内 Gamma 均值: {analysis['gamma_mean']:.4f}")
    else:
        print("无法生成推荐配置")


=== 007467 参数分析 ===

>>> 推荐配置:
    Season Length: 24
    Alpha: 0.0948
    Beta:  0.0000
    Gamma: 0.0532
    MSE:   0.000239

>>> 稳定性分析:
    分析窗口: end_day >= -100
    最常出现的 Season Length: 24
    该配置在窗口内出现次数: 2
    窗口内 Alpha 均值: 0.0945
    窗口内 Beta 均值: 0.0000
    窗口内 Gamma 均值: 0.0530

=== 004253 参数分析 ===

>>> 推荐配置:
    Season Length: 21
    Alpha: 0.1188
    Beta:  0.0048
    Gamma: 0.1545
    MSE:   0.000206

>>> 稳定性分析:
    分析窗口: end_day >= -100
    最常出现的 Season Length: 21
    该配置在窗口内出现次数: 1
    窗口内 Alpha 均值: 0.1188
    窗口内 Beta 均值: 0.0048
    窗口内 Gamma 均值: 0.1545


In [9]:
# 根据分析结果更新推荐参数
for code in optimizer.assets:
    asset = optimizer.get_asset(code)
    
    # 使用 analyze_stability 获取推荐配置
    recommended = asset.analyze_stability(recent_threshold=-100)
    
    if recommended:
        # 提取参数部分（不包括 analysis）
        params = {
            'season_length': recommended['season_length'],
            'alpha': recommended['alpha'],
            'beta': recommended['beta'],
            'gamma': recommended['gamma'],
            'mse': recommended['mse'],
        }
        
        # 更新资产的推荐参数
        asset.update_params(params)
        
        print(f"{code} 推荐参数已更新: {params}")
    else:
        print(f"{code} 无法生成推荐配置")

print("\n所有资产参数已更新完成")

# 重新获取汇总信息
hw_summary = optimizer.get_summary()

007467 推荐参数已更新: {'season_length': 24, 'alpha': 0.09484377970288167, 'beta': 0.0, 'gamma': 0.05321821517682557, 'mse': 0.00023851139500974443}
004253 推荐参数已更新: {'season_length': 21, 'alpha': 0.11884034505423396, 'beta': 0.004832728421863647, 'gamma': 0.15447024044944965, 'mse': 0.00020641441783075467}

所有资产参数已更新完成


In [11]:
# 查看更新后的汇总信息
for item in hw_summary:
    if item['status'] == 'success':
        print(f"{item['code']}: {item['params']}")

007467: {'alpha': 0.09484377970288167, 'beta': 0.0, 'gamma': 0.05321821517682557, 'season_length': 24, 'mse': 0.00023851139500974443}
004253: {'alpha': 0.11884034505423396, 'beta': 0.004832728421863647, 'gamma': 0.15447024044944965, 'season_length': 21, 'mse': 0.00020641441783075467}


In [12]:
optimizer.save_summary('./hw_opt_results', 'demo_hw')